In [1]:
import os
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
import json

np.random.seed(0)
random.seed(0)

# Data Preprocessing


In [2]:
def avgOutPoint1(df):
    #Change point 1 readings to Avg. of Point 2.
    label_list=list(df['label'])
    conc_list = list(df['AT400(CO2 %)'])
    p=0
    num=0
    i=0
    sampling2_avgcon = list()
    while i<len(label_list):
        if label_list[i]==2:
            p=p+conc_list[i]
            num=num+1
            i=i+1
        else:
            if p ==0 and num==0:
                i=i+1
                pass
            else:
                sampling2_avgcon.append(p/num)
                i =i+1
                num=0
                p=0

    i=0
    k=-1
    while i<(len(label_list)-1):
        if label_list[i]==1:
            conc_list[i]=sampling2_avgcon[k]
            i=i+1
        else:
            if label_list[i+1]==1 :
                k=k+1
                i=i+1
            else:
                i=i+1
    #set new AT400
    df['AT400(CO2 %)']=pd.Series(data=conc_list,index=df.index)
    return df

def columnSeparator(df):
    # linear interpolation to fill missing values
    for i in range(1,7): # 6 sampling points
        new_con =list()
        j = 0
        while j<df.shape[0]:
            if df.iloc[j]['label']==i:
                new_con.append( df.iloc[j]['AT400(CO2 %)'])
                j = j+1
            else:
                new_con.append(np.nan)
                j = j+1
        df[str(i)+"_sampling"]=pd.Series(data=new_con,index=df.index)
        df[str(i)+"_sampling"]=df[str(i)+"_sampling"].interpolate(method="linear")
    # Notice the df will contain NaN for the first few rows for each sampling point column
    return df

def getSampleSet(df_list, callback, train_feature_list, label_list):
    conc_label_list=[]
    for i in range(len(df_list)):
        # The 1st element in label list is label
        nset = df_list[i].shape[0]-callback
        print("Number of set: ", nset)
        conc_input=np.zeros((df_list[i].shape[0],6)) # Only the true reading is non-zero
        conc_set = df_list[i][label_list[1:]].values # Excl. 1st column, label column
        conc_set[np.isnan(conc_set)]=0
        parameter_set = df_list[i][train_feature_list].values
        for k in range(conc_input.shape[0]): # Only the true reading is non-zero, default for onehot
            conc_input[k][int(df_list[i]['label'][k])-1]=1        
        conc_input[np.isnan(conc_input)]=0
        parameter_set=np.hstack((parameter_set,conc_input))
        sample_set=np.zeros((nset,callback+1,parameter_set.shape[1]))
        label_set=np.zeros((nset,1,conc_set.shape[1]))
        for j in range(nset):
            sample_set[j]=parameter_set[0+j:callback+j+1]
            label_set[j]=conc_set[callback+j]
        
        if (i==0):
            total_sample_set=np.zeros((0,callback+1,parameter_set.shape[1]))
            total_label_set=np.zeros((0,1,conc_set.shape[1]))
        conc_label_list.append(label_set)
        total_sample_set = np.vstack([total_sample_set,sample_set])
        total_label_set = np.vstack([total_label_set,label_set])
        
    return total_sample_set, total_label_set, conc_label_list

In [3]:
data_root_path = './data'
data_path_list = glob.glob(os.path.join(data_root_path, "1*.xlsx"))
list_of_raw_df = []
test_set = '140207_1'
df_list = []
for i, path in enumerate(data_path_list):        
    xls = pd.ExcelFile(path)
    data_df = pd.read_excel(xls, sheet_name=0, index_col=0, header=[0,1])
    data_df.columns = data_df.columns.map(''.join)
    data_df=data_df.rename_axis('time').reset_index()
    tmp_name = list(data_df.columns)
    tmp_name[-1] = 'label'
    data_df.columns = tmp_name
    df_list.append(data_df)
    if test_set in path:
        test_idx = i
print("test_idx={}".format(test_idx))
print("test file is {}".format(data_path_list[test_idx]))

for i in range(len(df_list)):
    df_list[i] = df_list[i].set_index("time")
    df_list[i] = avgOutPoint1(df_list[i])
    if i == 0:
        origin_feature_num = df_list[i].shape[1]
        print("Origin input DF feature num(excl time, incl. label): ", origin_feature_num)
        
test_df_list = list(pd.Series(df_list)[[test_idx]]) # The 140207_1 is selected as the testset
train_df_list = list(pd.Series(df_list)[list(set(range(0,len(df_list),1))-set([test_idx]))])

for i in range(len(train_df_list)):
    if (i==0):
        tmp_full_values=train_df_list[i].values
        tmp_conc_values=train_df_list[i]['AT400(CO2 %)'].values
    else:
        tmp_full_values=np.concatenate((tmp_full_values, train_df_list[i].values), axis=0)
        tmp_conc_values=np.concatenate((tmp_conc_values, train_df_list[i]['AT400(CO2 %)'].values), axis=0)
tmp_full_values = tmp_full_values[:,:-1] # excl. label column as well
general_scaler = MinMaxScaler()
conc_scaler = MinMaxScaler()
general_scaler.fit(tmp_full_values)
conc_scaler.fit(tmp_conc_values.reshape(-1,1))
# general_max, general_min = general_scaler.data_max_, general_scaler.data_min_
conc_max, conc_min = conc_scaler.data_max_, conc_scaler.data_min_
print("concentration max, min: ", conc_max, conc_min)

# Process into + ['label', '1_sampling', '2_sampling', '3_sampling', '4_sampling', '5_sampling', '6_sampling']
for i in range(len(df_list)):
    df_list[i].iloc[:,:-1] = general_scaler.transform(df_list[i].iloc[:,:-1].values)
    df_list[i] = columnSeparator(df_list[i])
    df_list[i]=df_list[i].fillna(0)

# Separate df_list into Train & Test df lists
test_df_list = list(pd.Series(df_list)[[test_idx]]) # The 140207_1 is selected as the testset
train_df_list = list(pd.Series(df_list)[list(set(range(0,len(df_list),1))-set([test_idx]))])

print("train list: ", len(train_df_list))
label_list=['label', '1_sampling', '2_sampling', '3_sampling', '4_sampling', '5_sampling', '6_sampling']
train_feature_list = list(np.sort(list(set(df_list[0].columns)-set(label_list))))

test_idx=6
test file is ./data/140207_1.xlsx
Origin input DF feature num(excl time, incl. label):  91
concentration max, min:  [12.03671837] [0.]
train list:  7


# Save processed data into csv

In [4]:
# Save training DataFrames
root_path = 'processed_data'

if not os.path.exists(root_path):
    os.makedirs(root_path)
for i, df in enumerate(train_df_list):
    filename = f'{root_path}/train_data_{i}.csv'
    df.reset_index().to_csv(filename, index=False)
    print(f"Saved {filename}")

count = 0
# Save test DataFrame (since test_df_list contains only one DataFrame)
test_df_list[0].reset_index().to_csv(f'{root_path}/test_data.csv', index=False)
print(f"Saved {root_path}/test_data.csv")
    


Saved processed_data/train_data_0.csv
Saved processed_data/train_data_1.csv
Saved processed_data/train_data_2.csv
Saved processed_data/train_data_3.csv
Saved processed_data/train_data_4.csv
Saved processed_data/train_data_5.csv
Saved processed_data/train_data_6.csv
Saved processed_data/test_data.csv


# Save JSON file for train and test loader with specific window size

In [24]:

train_data = []
test_data = []
window_size = 50
count = 0
for i, df in enumerate(train_df_list):
    num_data = df.shape[0]
    num_windows = num_data - window_size
    for j in range(num_windows):
        train_data.append({})
        train_data[count]['filename'] = f'{root_path}/train_data_{i}.csv'
        train_data[count]['start_idx'] = j   
        count += 1 

count = 0
num_data = test_df_list[0].shape[0]
num_windows = num_data - window_size
for j in range(num_windows):
    test_data.append({})
    test_data[count]['filename'] = f'{root_path}/test_data.csv'
    test_data[count]['start_idx'] = j   
    count += 1 

# Save train_data and test_data as JSON files
train_json_filename = f'{root_path}/train_data_windows_{window_size}.json'
test_json_filename = f'{root_path}/test_data_windows_{window_size}.json'

# Save training data windows
with open(train_json_filename, 'w') as f:
    json.dump(train_data, f, indent=2)
print(f"Saved {train_json_filename} with {len(train_data)} windows")

# Save test data windows
with open(test_json_filename, 'w') as f:
    json.dump(test_data, f, indent=2)
print(f"Saved {test_json_filename} with {len(test_data)} windows")

# Also save metadata about the window configuration
metadata = {
    'window_size': window_size,
    'num_train_files': len(train_df_list),
    'num_test_files': len(test_df_list),
    'num_train_windows': len(train_data),
    'num_test_windows': len(test_data),
    'train_file_shapes': [df.shape for df in train_df_list],
    'test_file_shapes': [df.shape for df in test_df_list],
    'label_list': label_list,
    'train_feature_list': train_feature_list
}

metadata_filename = f'{root_path}/dataset_metadata_{window_size}.json'
with open(metadata_filename, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"Saved {metadata_filename}")

print(f"\nDataset Summary:")
print(f"Window size: {window_size}")
print(f"Total training windows: {len(train_data)}")
print(f"Total test windows: {len(test_data)}")
print(f"Training files: {len(train_df_list)}")
print(f"Test files: {len(test_df_list)}")

Saved processed_data/train_data_windows_50.json with 430 windows
Saved processed_data/test_data_windows_50.json with 68 windows
Saved processed_data/dataset_metadata_50.json

Dataset Summary:
Window size: 50
Total training windows: 430
Total test windows: 68
Training files: 7
Test files: 1
